# 1. Загрузка данных

Устанавливаем библиотеку для работы с facebook

In [1]:
!pip install facebook-sdk -q

Инициализируем facebook graph api и получаем информацию об исследуемой группе.

In [2]:
# Импорт библиотеки, инициализация токеном для авторизации.
import facebook
token = ''
graph = facebook.GraphAPI(access_token = token, version='2.3')

# Получаем id и название выбранной группы
group_name = 'SFSofUkraine'
group_info = graph.get_object(id = group_name)
group_info = {'name' : group_info['name'], 'id' : group_info['id']}

Получаем данные комментариев из выбранной группы.

In [17]:
# Библиотека для определения пола по имени
# !pip install genderize -q Необходимо раскоментировать и запустить 1 раз для установки библиотеки
from genderize import Genderize

# Функция для определения пола по имени
def getGender(name):
    return int (Genderize().get([name])[0]['gender'] == 'male')

In [18]:
# Функция для преобразования данных из формата facebook в более удобный для работы
def parseComment(comment_dict):
    comment = {}
    comment['message'] = comment_dict['message']
    comment['post_time'] = comment_dict['created_time']
    comment['likes'] = comment_dict['like_count']
    comment['author'] = comment_dict['from']['name']
    # Делим строку автор на имя и фамилия, проверяем какому полу наиболее вероятно принадлежит это имя
    comment['author_gender'] = getGender(comment['author'].split()[0])
    return comment
    

In [19]:
import requests

# Общая функция для сбора данных из facebook
def getGroupComments(group_id, pages_number):
    # Массив всех комментариев
    parsed_comments = []
    # Cчетчик страниц
    counter = 1
    
    # Получаем первоначальный список комментариев
    fb_data = graph.get_connections(id = group_id, connection_name='posts')
    
    while (True):
        print('Обрабатываем страницу: ', counter)
        counter += 1
        try:
            parsed_comments += parseGroupCommentsPage(fb_data)
        
            if (counter > pages_number):
                break
        
 
            fb_data = requests.get(fb_data['paging']['next']).json()
        # если нет следующей страницы - выходим из цикла
        except KeyError:
            break
        except:
            print('Достигнут лимит в 1000 имен в сутки на сервесе определения пола по имени')
            break
    print('Загрузка комментариев завершена')
    return parsed_comments

# Функция для сбора данных с одной страницы ответа API facebook
def parseGroupCommentsPage(fb_data):
    # Массив комментариев на странице
    comments = []
    # Проходим по всем комментариям на странице
    for data_element in fb_data['data']:
        # Проверяем есть ли комментарии у страницы
        if 'comments' in data_element:
            for comment in data_element['comments']['data']:
                # Проверяем есть ли текст в комментарии
                if (len(comment['message']) > 0) :
                    filtered_comment = parseComment(comment)
                    comments.append(filtered_comment)
    return comments

In [22]:
data = getGroupComments(group_info['id'], 3)

Обрабатываем страницу:  1
Обрабатываем страницу:  2
Обрабатываем страницу:  3
Загрузка комментариев завершена


Проверим полученные данные

In [23]:
# библиотека для удобного просмотра данных
import pandas as pd

# Показываем первые 5 строк
data = pd.DataFrame(data)
data.head()

,author,author_gender,likes,message,post_time
0,Igor Bocharov,1,1,За счет каких изменений в структуре и процесса...,2016-05-20T20:10:35+0000
1,Юля Машталер,0,2,"Их не сокращать нужно, а профессиональный уров...",2016-05-21T12:13:24+0000
2,Alexey Krivtsov,1,0,Открытость этим не измеряется,2016-05-22T07:14:06+0000
3,Denys Davydov,1,0,коли можна буде онлайн бачити отриманий прибут...,2016-05-20T12:40:57+0000
4,Andrey Skibnev,1,0,А откуда она?,2016-05-20T16:36:40+0000


# 2. Обработка данных

Подключаем необходимые библиотеки для обработки данных

In [24]:
# Установка библиотек, если их нет. Необходимо запустить 1 раз.
# !pip install pymorphy2 -q 
# !pip install stop_words -q 

from stop_words import get_stop_words # необходимо для удаления стоп слов
import pymorphy2 # библиотека для приведения слов к нормальной форме
import string # библиотека для работы со строками
from nltk.tokenize import TweetTokenizer # библиотека для разбиения текстов на слова

In [25]:
# Функция для разбития текста на слова
def split_text(text):
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(text) 

# Функция для приведения слов к нормальной форме.
def convert_to_normal_form(words_list):
    morph = pymorphy2.MorphAnalyzer()
    normal_forms_list = []
    for word in words_list:
        # работаем только со словами, не с пунктуацией
        if word not in string.punctuation and word[0] != "<":
            norm_form = morph.parse(word)[0].normal_form # получаем нормальную форму слова
            normal_forms_list.append(norm_form) # и добавляем ее в массив
    return normal_forms_list

# Общая функция обработки текста
def convert_text(text):
    words_list = split_text(text) # разбиваем текст на слова
    norm_words_list = convert_to_normal_form(words_list) # приводим слова к нормальной форме
    filtered_words = [w for w in norm_words_list if not w in get_stop_words('ukrainian')] # удаляем стоп слова на украинском
    filtered_words = [w for w in filtered_words if not w in get_stop_words('russian')] # удаляем стоп слова на русском
    return " ".join(filtered_words) # соединяем слова обратно в текст

Обработаем полученный текст

In [26]:
data['message'] = data.apply(lambda row: convert_text(row['message']), axis=1)

Посмотрим текст после обработки:

In [27]:
data.head()

,author,author_gender,likes,message,post_time
0,Igor Bocharov,1,1,счёт изменение структура процесс происходить с...,2016-05-20T20:10:35+0000
1,Юля Машталер,0,2,сокращать профессиональный уровень повышать,2016-05-21T12:13:24+0000
2,Alexey Krivtsov,1,0,открытость измеряться,2016-05-22T07:14:06+0000
3,Denys Davydov,1,0,можный онлайн бачить отримания прибутка даний ...,2016-05-20T12:40:57+0000
4,Andrey Skibnev,1,0,откуда,2016-05-20T16:36:40+0000


# 3. Сохранение полученных данных в файл

In [28]:
data.to_excel('comments.xlsx')